### CNN for DDoS intrusion detection

In [2]:
import numpy as np
import pandas as pd

from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Sequential

from keras import callbacks
from keras.callbacks import CSVLogger

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from sklearn import accuracy_score, recall_score, precision_score, f1_score
tf.logging.set_verbosity(tf.logging.ERROR)

Read and prepare dataset

In [4]:
np.random.seed(667)  # randomize

testdata = pd.read_csv("DDoS2007_spounged.csv", header=None)


#X = traindata.iloc[:,0:42]
#Y = traindata.iloc[:,0]
y_ini = testdata.iloc[:,0]
X_ini = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X_ini)
X_tras = scaler.transform(X_ini) #X Transformed

# reshape input to be [samples, time steps, features]
X = np.reshape(X_tras, (X_tras.shape[0],X_tras.shape[1],1))
y = np.array(y_ini)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

CNN Structure

In [8]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=3, strides=1, padding="same",activation="relu",input_shape=(41, 1)))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=(2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

Training (FIT)

In [ ]:
cnn.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

Train on 700000 samples, validate on 300001 samples
Epoch 1/3
128640/700000 [====>.........................] - ETA: 7:30 - loss: 0.5208 - acc: 0.7452

Results

In [ ]:
y_prob = cnn.predict(X_test)
y_pred = (y_prob > 0.5).astype(np.int)
np.savetxt("cnn.txt", y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")

print("confusion matrix")
print("----------------------------------------------")
print("accuracy")
print("%.4f" %accuracy)
print("recall")
print("%.4f" %recall)
print("precision")
print("%.4f" %precision)
print("f1score")
print("%.4f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("==============================================")